# MovieLens database cleaning

## Extract needed dates

According to its ReadMe, this database has records from ratings dating from between January 09, 1995 and March 31, 2015. The Netflix database only ranges from October 1998 to December 2005, so that many entries are useless.

The numbers of distinct users goes from 138,493 to 52,875.

In [ ]:
import pandas as pd
import numpy as np
from datetime import datetime
import time

In [ ]:
db = pd.read_csv("data/MovieLens/ratings.csv", header=None, encoding="UTF-8", names=["userId","movieId","rating","timestamp"])
print(db.shape)

# Create the start and stop boundaries for the timestamp
# format: seconds elapsed since 1st Jan 1970
start_date =  time.mktime(datetime(1998,10,1).timetuple())
end_date =  time.mktime(datetime(2006,1,1).timetuple())

# Remove out-of-bounds entries
a = (db["timestamp"] < start_date) 
print("There are", sum(a), "reviews made before the 1st October 1998.")
b = (db["timestamp"] > end_date) 
print("There are", sum(b), "reviews made after the 31st December 2005.")

db.head()

In [ ]:
# Get number of distincts users
list_ml_movies = db.movieId.unique()
print("There are", len(db.userId.unique()), "users left.")
print("There are", len(db.movieId.unique()), "movies left.")

## Remove not matching movies

Only the movies that are present in both datasets need to be kept, the other ones would not be taken into account in the scoring functio anyway. 

In the process, also make the movieIDs consistent across the two databases so that the titles do not need to be processed anymore.


In [ ]:
# Extract movie titles
import csv

# These titles are present several times and cannot be uniquely identified
wrong_movies = ["pinocchio(2002)", "lastmanstanding(1996)", "emma(1996)",
                "hamlet(2000)", "hamlet(1990)", "menwithguns(1997)",
                "thehunchbackofnotredame(1999)","thelucyshow(1962)",
                "stranded(2002)","frankenstein(2004)","secondskin(2000)",
                "elvira'shorrorclassics(1959)","waroftheworlds(2005)",
                "chaos(2005)","offside(2006)","blackout(2007)","thegirl(2012)",
                "crimewave(1985)","20,000leaguesunderthesea(1997)",
                "aladdin(1992)","beneath(2013)","thedisappeared(2008)",
                "paradise(2013)","clearhistory(2013)","casanova(2005)",
                "johnnyexpress(2014)","darling(2007)"]
nf_movies = {};
with open('data/Netflix/movie_titles.txt', encoding="ISO-8859-1") as nf:
    for col1,col2,*col3 in csv.reader(nf):
        #s = ''.join(col3).lower().replace(" ","")
        #s = s[:(s.find('(') if s.find('(') !=-1 else len(s))]
        s = (''.join(col3)+"("+col2+")").lower().replace(" ","")
        if s not in wrong_movies:
            nf_movies[s] = int(col1)
nf_movies = pd.Series(nf_movies, name='Netflix')

ml_movies = {};
with open('data/MovieLens/movies.csv', encoding="UTF-8") as ml:
    for col1,col2,*col3 in csv.reader(ml):
        if int(col1) in list_ml_movies:
            s = ''.join(col2).lower().replace(" ","")
            loc = s.find(",the(")
            if loc !=-1:
                s = "the" + s[:loc] + s[loc+4:]
            else:
                loc=s.find(",a(")
                if loc != -1:
                    s = "a" + s[:loc] + s[loc+2:] 
                else:
                    loc=s.find(",an(")
                    if loc != -1:
                        s = "an" + s[:loc] + s[loc+3:]
            if s not in wrong_movies:
                ml_movies[s] = int(col1)

ml_movies = pd.Series(ml_movies, name='MovieLens')
print(len(nf_movies))
print(len(ml_movies))

In [ ]:
# Find intersection of movie lists using title+date
common_movies = nf_movies.index.intersection(ml_movies.index)
matches = pd.Series(data=ml_movies.loc[common_movies].values,index=nf_movies.loc[common_movies].values)
matches.name = 'ml_movieId'
matches.index.name = 'nf_movieId'
print('There is', matches.shape[0], 'matches')
display(matches.head())

In [ ]:
# Discard non matches
db = db.loc[db['movieId'].isin(matches),:]

In [ ]:
z = []
#df = pd.DataFrame(columns=["userId","movieId","rating","timestamp"])
for i in matches.index:
    tmp = pd.read_csv("data/Netflix/training_set/mv_" + format(i,'07d')+".txt", header=None,
                         names=["userId","rating","timestamp"],encoding="ISO-8859-1")
    tmp["movieId"] = matches[i]
    z.append(tmp)
df = pd.concat(z,copy=False)

In [ ]:
# Sanity check
assert db.movieId.unique().shape[0] == df.movieId.unique().shape[0] == matches.shape[0]

In [ ]:
df.to_csv("DF.csv",";", index=False)

In [ ]:
db.to_csv("DB.csv",";", index=False)